In [121]:
%cd /home/richard_fernandes/Downloads/sentinelsat-main

/home/richard_fernandes/Downloads/sentinelsat-main


1.  Parse csv file to extract ALL image names, even multiple entries
        parse data as csv
        extract the column as a list of lists
        flatten the lixt of lists into one big list
        remove null none 
2.  make pandas df from clean list with one column correspondng to input image name
3.  write function to search for the input image name in gee and return system:id , put in new column of pandas df
        start gee api
        in python for loop of df column of image names
            use gee command to filter the S2 collection to get image 
            real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index","20210719T190919_20210719T191700_T10TER")) \
                         .first() \
                         .get('PRODUCT_ID') \
                         .getInfo()
4.  iterate over list of real names
    

In [122]:
#import libraries
import ee
import pandas as pd
import os

# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [123]:
#Autheticate and initillize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWtgzh5lUd0y1q2J01oMGOIS2Mnn7_Elf9egqdey0yqD8LmYt8is_4PKtGM



Successfully saved authorization token.


In [125]:
def directory1(directory):
    """Gets the names of a directory"""
    entries = os.listdir(directory)
    return entries
    # print(entries)  

In [170]:
def list_splitter(data):
    """cleans/splits df of list of images"""
    
    # Input data
    df = pd.read_csv(data)
    # print(df)
    
    # Change type to string, remove [], send to list
    df['Images'] = df['Images'].astype(str)
    # print(df['Images'])
    df['Images'] = df['Images'].map(lambda x: x.lstrip('[').rstrip(']'))
    list_of_img =  df["Images"].tolist() 
    # print(list_img)
    
    # Clean list without nan
    cleanedList = [x for x in list_of_img if str(x) != 'nan']
    
    # Print list
    # print("list1")
    # print(cleanedList)
    
    # print("#---------------------------")
    # print(len(cleanedList))
    
    # Arrays
    cleanedList2  = []
    list_to_add = []
    
    # Find list images containing more than 1 img
    for counter0 in range(0,len(cleanedList)):
        count_comma = cleanedList[counter0].count(',')
        # print(count_comma)
       
        if count_comma>0:
            list_multiple  = cleanedList[counter0]
            # print(list_multiple)
            cleanedList2.append(list_multiple)
            
    # Find list images containing 1 img  
    cleanedList3 = [item for item in cleanedList if item not in cleanedList2]
    
    # Print
    # print("clean list2")
    # print(cleanedList2)
    # print("clean list3")
    # print(cleanedList3)

    # Split the data into separate imgs 
    for counter1 in range(0,len(cleanedList2)):
        list_item =cleanedList2[counter1] 
        # print(list_item)
        mylist = list_item.split(',')
        for counter2 in range(0,len(mylist)):
            withoutspace  = mylist[counter2].replace(" ","")
            
            list_to_add.append(withoutspace)
    
    
    # print("list to add")
    # print(list_to_add)
    
    
    # add two lists      
    list_fin = list_to_add+cleanedList3
    
    # create dataframe
    newdf= pd.DataFrame(list_fin,columns=["Images__clean"])
    newdf = newdf.drop_duplicates()

    # newdf.to_csv("/home/richard_fernandes/Downloads/data1.csv")
    
    print("______________________________________________")
    return newdf

    
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/ABBY_kmlwithImages.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/DSNYCLEARIMAGES2.csv"))
# print(list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/UNDECLEARIMAGES.csv"))

______________________________________________
                            Images__clean
0  20190606T165901_20190606T170333_T15TYM
1  20190606T165901_20190606T170333_T16TCS
2  20200806T164901_20200806T165818_T15TYM
3  20200806T164901_20200806T165818_T16TCS


In [103]:
def real_name(short_name):
    """ function to help find real names of files"""
    real_name = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                    .filter(ee.Filter.equals("system:index",short_name)) \
                    .first() \
                    .get('PRODUCT_ID') \
                    .getInfo()

    return real_name

In [40]:
# print(real_name("20210719T190919_20210719T191700_T10TER"))

S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339


In [83]:
def search_string(string1):
    """ searchs esa datahub for image and downsloads it """
    
    # Connect to the API
    api = SentinelAPI('****', '*******', 'https://apihub.copernicus.eu/apihub')

    # Split string 
    string = string1 
    split_s=string.split("_")
    
    # Built a file string
    sat_name = "S2A"+"_"
    rest_name = split_s[1]+"_"+split_s[2]+"_"+split_s[3]+"_"+split_s[4]+"_"+split_s[5]+"_"+split_s[6]
    safe_name = '.SAFE'
    # # wc = '_*_'

    #try and except to find errors
    try:
        print("#--------Try---------------------------------------")
        sat_name = 'S2A_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)

        query1 = api.query(filename = filename1)
        # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)
        # print(result)
        
        #check if something is downloaded
        if str(result) =="ResultTuple(downloaded={}, retrieval_triggered={}, failed={})":
            raise ValueError('Nothing was dowloaded')

    except:
        print("#--------Except---------------------------------------")
        sat_name ='S2B_'
        
        # Find and query image
        filename1 = sat_name+rest_name+safe_name
        # print(filename1)
        
        query1 = api.query(filename = filename1)
        # # print(query1)
        
        # download information
        api.download_all(query1)
        result = api.download_all(query1)

    # print(result)
# 
# print(search_string("S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339"))

#--------Try---------------------------------------
#--------Except---------------------------------------


ResultTuple(downloaded={'1d58d0fd-108c-424c-a818-43d12545e305': {'id': '1d58d0fd-108c-424c-a818-43d12545e305', 'title': 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339', 'size': 741432504, 'md5': '7c8c1c3c7358bd01a8bd71627d69e9f3', 'date': datetime.datetime(2021, 7, 19, 19, 9, 19, 24000), 'footprint': 'POLYGON((-122.0253 46.04752322186573,-122.03458 46.02596045913228,-122.09686 45.88072405579696,-122.15697 45.73506781249301,-122.216995 45.589502397244495,-122.27751 45.44413955351746,-122.337585 45.2987002868348,-122.39809 45.15339736104352,-122.4362 45.061817282498176,-123.00026 45.06525999447365,-123.00026 46.05357436995249,-122.0253 46.04752322186573))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('1d58d0fd-108c-424c-a818-43d12545e305')/$value", 'Online': True, 'Creation Date': datetime.datetime(2021, 7, 20, 1, 18, 17, 594000), 'Ingestion Date': datetime.datetime(2021, 7, 20, 1, 18, 0, 554000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odat

In [ ]:
# List of csvs in directory
list_directory = directory1("/home/richard_fernandes/Downloads/S2_clearskyImage")
# print(list_directory)



df = pd.DataFrame()

# Looping through csvs
for counter0 in range(0,len(list_directory)): 
# for counter0 in range(0,2):  

    # Clean lists of imgs in csv
    newdf = list_splitter("/home/richard_fernandes/Downloads/S2_clearskyImage/"+list_directory[counter0])
    # print(newdf)
    
    # Search every Image
    for counter in range(0,len(newdf.index)): 
        if ~newdf.empty:
            
            # Run through process
            # print("p")
            
            df = pd.concat([df, newdf])
           
            # Takes img name
            # name_stage0 = newdf['Images__clean'][counter]

            # print(name_stage0)
            
            # Finds real name of img
            # name_stage1 = real_name(name_stage0)
            
            # search and download img
            # search_string(name_stage1)
            # # print(name_stage1)
            
            
#getrid of duplicate and reset index
df = df.drop_duplicates()
print(df)
df = df.reset_index()

# df.head()
df.to_csv("/home/richard_fernandes/Downloads/Images_clean.csv")

#loop for finding
for counter in range(0,len(df.index)):
    name_stage0 = df['Images__clean'][counter]
    # print(df['Images__clean'][counter])
     
    # Finds real name of img
    name_stage1 = real_name(name_stage0)
            
    # search and download img
    search_string(name_stage1)
    # # print(name_stage1)



______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
______________________________________________
                             Images__clean
0   20190606T165901_20190606T170333_T15TYM
1   20190606T165901_20190606T170333_T16TCS
2   20200806T164901_20200806T165818_T15TYM
3   20200806T164901_20200806T165818_T16TCS
0   20200627T173909_20200627T174744_T13TGM
1   20200627T173909_20200627T174744_T14TLS
2   20210612T173909_20210612T174342_T13TGM
3   20210612T173909_20210612T174342_T14TLS
0 

#--------Except---------------------------------------
ResultTuple(downloaded={}, retrieval_triggered={}, failed={})
#--------Try---------------------------------------


#--------Except---------------------------------------
ResultTuple(downloaded={}, retrieval_triggered={}, failed={})
#--------Try---------------------------------------


#--------Except---------------------------------------
ResultTuple(downloaded={}, retrieval_triggered={}, failed={})
#--------Try---------------------------------------



MD5 checksumming:   0%|                                                          | 0.00/1.13G [00:00<?, ?B/s]
MD5 checksumming:   5%|██▎                                               | 53.6M/1.13G [00:00<00:02, 536MB/s]
MD5 checksumming:   9%|████▊                                              | 107M/1.13G [00:00<00:01, 520MB/s]
MD5 checksumming:  14%|███████▎                                           | 161M/1.13G [00:00<00:01, 529MB/s]
MD5 checksumming:  19%|█████████▋                                         | 214M/1.13G [00:00<00:01, 520MB/s]
MD5 checksumming:  24%|████████████                                       | 268M/1.13G [00:00<00:01, 526MB/s]
MD5 checksumming:  28%|██████████████▍                                    | 320M/1.13G [00:00<00:01, 510MB/s]
MD5 checksumming:  33%|████████████████▊                                  | 372M/1.13G [00:00<00:01, 472MB/s]
MD5 checksumming:  37%|███████████████████                                | 422M/1.13G [00:00<00:01, 482MB/s]
MD5 check

...

LTA retrieval: 100%|████████████████████████████████████████████████████| 1/1 [21:10<00:00, 1270.50s/product]


MD5 checksumming:   0%|                                                          | 0.00/1.16G [00:00<?, ?B/s]
MD5 checksumming:   4%|██▏                                               | 51.9M/1.16G [00:00<00:02, 519MB/s]
MD5 checksumming:   9%|████▌                                              | 104M/1.16G [00:00<00:02, 506MB/s]
MD5 checksumming:  13%|██████▊                                            | 154M/1.16G [00:00<00:02, 499MB/s]
MD5 checksumming:  18%|█████████                                          | 206M/1.16G [00:00<00:01, 506MB/s]
MD5 checksumming:  22%|███████████▎                                       | 257M/1.16G [00:00<00:01, 502MB/s]
MD5 checksumming:  27%|█████████████▌                                     | 307M/1.16G [00:00<00:01, 499MB/s]
MD5 checksumming:  31%|███████████████▊                                   | 359M/1.16G [00:00<00:01, 505MB/s]
MD5 chec

ResultTuple(downloaded={'ae6385a6-380b-48e8-adfc-532dea0c5a5d': {'id': 'ae6385a6-380b-48e8-adfc-532dea0c5a5d', 'title': 'S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901', 'size': 1156811416, 'md5': '8d230338edab31cf3ce44d8147d8a685', 'date': datetime.datetime(2019, 8, 26, 15, 38, 19, 24000), 'footprint': 'POLYGON((-72.53372 43.32625697177811,-71.181885 43.28908179915951,-71.24213 42.30245161555175,-72.57269 42.33837210189082,-72.53372 43.32625697177811))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('ae6385a6-380b-48e8-adfc-532dea0c5a5d')/$value", 'Online': True, 'Creation Date': datetime.datetime(2019, 8, 27, 0, 28, 18, 325000), 'Ingestion Date': datetime.datetime(2019, 8, 27, 0, 27, 48, 564000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('ae6385a6-380b-48e8-adfc-532dea0c5a5d')/Products('Quicklook')/$value", 'path': 'S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901.zip', 'downloaded_bytes': 0}}, retrieval_triggere

LTA retrieval: 100%|██████████████████████████████████████████████████| 1/1 [2:20:01<00:00, 8401.96s/product]


MD5 checksumming:   0%|                                                          | 0.00/1.07G [00:00<?, ?B/s]
MD5 checksumming:   4%|██                                                | 44.0M/1.07G [00:00<00:02, 440MB/s]
MD5 checksumming:   8%|████▏                                             | 90.6M/1.07G [00:00<00:02, 456MB/s]
MD5 checksumming:  13%|██████▍                                            | 136M/1.07G [00:00<00:02, 447MB/s]
MD5 checksumming:  17%|████████▋                                          | 181M/1.07G [00:00<00:02, 435MB/s]
MD5 checksumming:  21%|██████████▋                                        | 224M/1.07G [00:00<00:01, 427MB/s]
MD5 checksumming:  26%|█████████████▍                                     | 281M/1.07G [00:00<00:01, 474MB/s]
MD5 checksumming:  31%|███████████████▉                                   | 335M/1.07G [00:00<00:01, 494MB/s]
MD5 chec

ResultTuple(downloaded={'9ea74920-3fe7-446a-a212-3cde2b5dbd12': {'id': '9ea74920-3fe7-446a-a212-3cde2b5dbd12', 'title': 'S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238', 'size': 1069124694, 'md5': 'bb7a71d0d3a19ecaed7eac6974b0bb4a', 'date': datetime.datetime(2019, 8, 22, 17, 39, 9, 24000), 'footprint': 'POLYGON((-107.14909 33.339957812432836,-107.13722 33.38423202275722,-106.875824 33.34337650900327,-106.87575 33.34369495338371,-106.875534 33.34366367143568,-106.87547 33.343924230987106,-106.875244 33.343888826485035,-106.87518 33.34410069884611,-106.87479 33.344039109838164,-106.874695 33.34440210881251,-106.87439 33.3443554535683,-106.87439 33.344359820574965,-106.8743 33.34434617185632,-106.87425 33.34451150454458,-106.874054 33.34447962925314,-106.87399 33.344684441124066,-106.87341 33.34459432969781,-106.87329 33.34501377617908,-106.87326 33.34500742141255,-106.87317 33.34534812081744,-106.87283 33.34529897862927,-106.87276 33.34557914644318,-106.87259 33.34555258492

LTA retrieval:   0%|                                                              | 0/1 [00:00<?, ?product/s]